In [1]:
FILES = [
    "grandmaster_nl_pl_only_plot.json",
    # "master_nl_pl_only_plot.json",
    # "expert_nl_pl_only_plot.json",
]

CLASS_NAME = {
    "grandmaster": "GrandMasterCode",
    "master": "MasterCode",
    "expert": "ExpertCode",
}

**Weaviate instance**

In [2]:
import weaviate
import json


weaviate_client = weaviate.Client("http://202.151.177.149:81")  # Replace with your endpoint
some_objects = weaviate_client.data_object.get()
if (json.dumps(some_objects)):
    print(True)
else:
    print(False)

print(json.dumps(some_objects))


True
{"deprecations": null, "objects": [{"class": "GrandMasterCode", "creationTimeUnix": 1680794367435, "id": "002bd101-b16d-48c7-a401-44c673520d1a", "lastUpdateTimeUnix": 1680794367435, "properties": {"code": "# Reference: https://www.kaggle.com/code/tarunpaparaju/champs-competition-chemistry-background-and-eda\n\nplt.figure(figsize=(26, 24))\nfor i, col in enumerate(typelist):\n    plt.subplot(4,2, i + 1)\n    sns.distplot(dipole_moments[train['type']==col]['X'],color = 'orange', kde=False)\n    sns.distplot(dipole_moments[train['type']==col]['Y'],color = 'red', kde=False)\n    sns.distplot(dipole_moments[train['type']==col]['Z'],color = 'blue', kde=False)\n    plt.title(col)"}, "vectorWeights": null}, {"class": "GrandMasterCode", "creationTimeUnix": 1680792753015, "id": "00300e91-5c0a-4237-9312-85a5c14ea286", "lastUpdateTimeUnix": 1680792753015, "properties": {"code": "import pandas as pd\nimport matplotlib.pyplot as plt\nimport networkx as nx\nimport nltk\nfrom nltk.util import ngr

**Elasticsearch Instance**

In [18]:
from elasticsearch import Elasticsearch
from json import loads

elastic_client = Elasticsearch("http://202.151.177.154:9200")

response = str(elastic_client.info())
print(response)

{'name': 'node40937-typhon-server.th1.proen.cloud', 'cluster_name': 'typhon-ir-cluster', 'cluster_uuid': 'VE76ILFVQhuoZlkd6KnPbw', 'version': {'number': '8.6.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'f67ef2df40237445caa70e2fef79471cc608d70d', 'build_date': '2023-01-04T09:35:21.782467981Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [46]:
# Example Obj
# {
#     "markdown": [Array<Str>],
#     "processed": [Array<Str>],
#     "code": Str
# }

# Example Weaviate Classname
# 1. GrandMasterCode
# 2. MasterCode
# 3. ExpertCode

# ElasticSearch Query Example
# baseQuery = {
#     "query": {
#         "match": {
#             "markdown": {
#                 "query": markdown_desc
#             }
#         }
#     }
# }
# processedQuery = {
#     "query": {
#         "match": {
#             "processed": {
#                 "query": markdown_desc
#             }
#         }
#     }
# }

def getMLRecommendation(text: str) -> dict:
    # md_text = obj['markdown']
    cur_class = "grandmaster"
    near_text = {"concepts": [text]}
    fetched = (weaviate_client.query
                      .get(CLASS_NAME[cur_class], ["code"])
                      .with_near_text(near_text)
                      .with_limit(1)
                      .do()
                      )
    data = fetched['data']['Get'][CLASS_NAME[cur_class]]
    return data[0]

def getElasticQuery(query, type=["base", "processed"]):
    if (type == "processed"):
        result = {
        "query": {
            "match": {
                "processed": {
                    "query": query
                }
            }
        }
    }
    else:
        result = {
        "query": {
            "match": {
                "markdown": {
                    "query": query
                }
            }
        }
    }
    return result 

def getElasticRecommendation(index, queryBody):
    response = elastic_client.search(index=index, body=queryBody)

    if response and response["hits"]["hits"]:
        result = response["hits"]["hits"][0]["_source"]
        return result['code']
    return []
    
    

In [ ]:
with open("grandmaster_nl_pl_only_plot.json") as f:
    grand_master_json = loads(f.read())

# markdown_desc = "create a scatter plot"
i = 1

for pair in grand_master_json:
    markdown_desc = "".join(pair["markdown"])


    response = elastic_client.search(index="grandmaster", body=baseQuery)

    if response and response["hits"]["hits"]:
        a = response["hits"]["hits"][0]
        # print(json.dumps(a, indent=4))
        res = response["hits"]["hits"][0]["_source"]

    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    print(json.dumps(res, indent=4))

    i = i + 1
    if (i == 10):
        i = 1
        break
# print("Hello World")

In [ ]:
testing_accumulate = []
# i = 0
for file_name in FILES:
    with open(file_name, 'r') as file:
        data = json.load(file)

        data_length = len(data)
        data_count = 1

        for row in data:
            rows_length = len(row['markdown'])
            row_count = 1

            for markdown in row['markdown']:
                temp_result = {
                    "original_md": markdown,
                    "original_code": row['code'],
                    "count": f'{data_count}-{row_count}/{rows_length}',
                    "data_rank": file_name[:-21],
                }
                
                # Get ML Recommendation
                recommended_code_ml = getMLRecommendation(markdown)
                temp_result["recommended_code"] = recommended_code_ml['code']
                temp_result["model"] = "machine-learning"

                # Get Elastic Recommendation
                query_base = getElasticQuery(markdown)
                query_base = getElasticQuery(markdown, "processed")
                recommended_code_es = getElasticRecommendation()

                # Append to the collection
                testing_accumulate.append(temp_result)

                print(f'Row Count: {row_count}/{rows_length}')
                row_count = row_count + 1

            print(f'Data Count: {data_count}/{data_length}')
            data_count = data_count + 1


In [63]:
print(json.dumps(testing_accumulate, indent=4))

[
    {
        "original_md": "# Acknowledgements\n\n1. [M5 Forecasting - Starter Data Exploration](https://www.kaggle.com/robikscube/m5-forecasting-starter-data-exploration) ~ by Rob Mulla\n2. [EDA and Baseline Model](https://www.kaggle.com/rdizzl3/eda-and-baseline-model) ~ by RDizzl3\n3. [How to Create an ARIMA Model for Time Series Forecasting in Python](https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/) ~ by Machine Learning Mastery\n4. [7 methods to perform Time Series forecasting (with Python codes)](https://www.analyticsvidhya.com/blog/2018/02/time-series-forecasting-methods/) ~ by Analytics Vidhya\n5. [Economics for the IB Diploma](https://www.cambridge.org/core/books/economics-for-the-ib-diploma/1918CF16A8FC979AAB19951A487DCB1C) ~ by Ellie Tragakes\n6. [Prophet](https://facebook.github.io/prophet/) ~ by Facebook",
        "recommended_code": "import numpy as np\nimport pandas as pd\nimport matplotlib.pylab as plt\nimport cv2\nimport seaborn as 

In [ ]:
app.get("/", async (req, res) => {
    const grandMasterJson = JSON.parse(
    await readFile('path/to/file.json')
    )

    grandMasterJson.map(async (pair) => {
        const markdownDesc = pair.markdown.reduce((acc, cur) => acc + cur, "");
        const response = await fetch(URL + "/api/v1/ir/base", {
            method: "POST", 
            headers: {
                "Content-Type": "application/json",
            },
            body: JSON.stringify({
                query: markdownDesc,
                index: "grandmaster",
            }),
        });

        const responseJson = await response.json();

        if(responseJson && responseJson.hits.hits.length > 0) {
            console.log(responseJson.hits.hits[0]);
        }
        console.log(responseJson)
    });

    res.status(200).send("Hello World")
})

create a scatter plot

In [ ]:
# Reference: https://www.kaggle.com/code/arthurtok/interactive-porto-insights-a-plot-ly-tutorial

# Scatter plot 
trace = go.Scatter(
    y = rf.feature_importances_,
    x = features,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        #size= rf.feature_importances_,
        #color = np.random.randn(500), #set color equal to a variable
        color = rf.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = features
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')